In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime, timedelta
from __future__ import division

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
pyoff.init_notebook_mode()

ModuleNotFoundError: No module named 'pandas'

## Data Wrangling

In [ ]:
#read the data

df = pd.read_excel("data/Online_Retail.xlsx", sheet_name='Online Retail')
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

#### Monthly Revenue

In [ ]:
# Converting from string to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#creating YearMonth field for the ease of reporting and visualization
df['InvoiceYearMonth'] = df['InvoiceDate'].map(lambda date: 100*date.year + date.month)

#calculate Revenue for each row and create a new dataframe with YearMonth - Revenue columns
df['Revenue'] = df['UnitPrice'] * df['Quantity']
df_revenue = df.groupby(['InvoiceYearMonth'])['Revenue'].sum().reset_index()
df_revenue.head(15)

In [ ]:
#Visualization
#X and Y axis inputs for Plotly graph. We use Scatter for line graphs

plot_df = [go.Scatter(x = df_revenue['InvoiceYearMonth'],
                      y = df_revenue['Revenue'],
                     )]

plot_layout = go.Layout(xaxis= {"type": "category"},
                        title = "Monthly Revenue"
                       )

fig = go.Figure(data = plot_df, layout= plot_layout)
pyoff.iplot(fig)

### Monthly Revenue Growth Rate

In [ ]:
#using pct_change() function to see monthly percentage change

df_revenue['MonthlyGrowth'] = df_revenue['Revenue'].pct_change()

#showing first 5 rows
df_revenue.head(5)

In [ ]:
#visualization - line graph

plot_df_monthlyGrowth = [go.Scatter( x = df_revenue.query("InvoiceYearMonth < 201112")["InvoiceYearMonth"],
                                      y = df_revenue.query("InvoiceYearMonth < 201112")["MonthlyGrowth"],
                                     )]

plot_layout_monthlyGrowth = go.Layout(xaxis = {"type": "category"},
                                     title = "Monthly Growth Rate")

fig2 = go.Figure(data = plot_df_monthlyGrowth, layout= plot_layout_monthlyGrowth)
pyoff.iplot(fig2)


In [ ]:
df_country = df['Country'].unique()
df_country

In [ ]:
# What Happened in April 
# Check Monthly Active customers

#creating a new dataframe with UK customers only
df_uk = df.query("Country == 'United Kingdom'").reset_index(drop = True)

#creating monthly active customers dataframe by counting unique Customer IDs
df_montly_active = df_uk.groupby('InvoiceYearMonth')['CustomerID'].nunique().reset_index()

df_montly_active


In [ ]:
# Plotting the Monthly Active customers

plot_df_AMC = [go.Bar(x = df_montly_active['InvoiceYearMonth'],
                     y = df_montly_active['CustomerID'],
                     )]
plot_df_layout = go.Layout(xaxis = {"type": "category"},
                          title = "Monthly Active Customers")

fig3 = go.Figure(data=plot_df_AMC, layout = plot_df_layout )

pyoff.iplot(fig3)

In [ ]:
# What Happened in April 
# Check Monthly Order Count

#creating Monthly Order Count dataframe for no. of oder by using quantity field

df_monthly_sales = df_uk.groupby('InvoiceYearMonth')['Quantity'].sum().reset_index()

df_monthly_sales

In [ ]:
# Plotting the Monthly Order Count

plot_df_monthly_sales = [go.Bar(x = df_monthly_sales['InvoiceYearMonth'],
                               y = df_monthly_sales['Quantity'],
                               )]

plot_monthly_sales_layout = go.Layout(xaxis = {"type": "category"},
                                     title = 'Monthly Total No. of Order')

fig4 = go.Figure(data = plot_df_monthly_sales, layout= plot_monthly_sales_layout )

pyoff.iplot(fig4)

In [ ]:
# What Happened in April 
#Average Revenue per Order

# create a new dataframe for average revenue by taking the mean of it

df_monthly_order_avg = df_uk.groupby('InvoiceYearMonth')['Revenue'].mean().reset_index()

df_monthly_order_avg.head(4)

In [ ]:
plot_monthly_order_avg = [go.Bar( x = df_monthly_order_avg['InvoiceYearMonth'],
                                y = df_monthly_order_avg['Revenue'],
                                )]

plot_layout_monthly_order_avg = go.Layout(xaxis = {"type": "category"},
                                         title = "Monthly Order Avarage")

fig5 = go.Figure(data =plot_monthly_order_avg , layout =plot_layout_monthly_order_avg)

pyoff.iplot(fig5)

In [ ]:
#create a df contaning CustomerID and first purchase date

df_min_purchase = df_uk.groupby('CustomerID').InvoiceDate.min().reset_index()
df_min_purchase.columns = ['CustomerID', 'MinPurchaseDate']
df_min_purchase['MinPurchaseYearMonth'] = df_min_purchase['MinPurchaseDate'].map(lambda date: 100*date.year + date.month)

#merge first purchase date column to our main dataframe (df_uk)

df_uk = pd.merge(df_uk, df_min_purchase, on='CustomerID')

df_uk.head()



In [ ]:
#create a column called User Type and assign Existing 
#if User's First Purchase Year Month before the selected Invoice Year Month

df_uk['UserType'] = "New"
df_uk.loc[df_uk['InvoiceYearMonth'] > df_uk['MinPurchaseYearMonth'], 'UserType'] = 'Existing'
#df_uk.head(5)

#calculate the Revenue per month for each user type
df_user_monthly_revenue = df_uk.groupby(['InvoiceYearMonth', 'UserType'])['Revenue'].sum().reset_index()

#filtering the dates and plot the result
df_user_monthly_revenue = df_user_monthly_revenue.query("InvoiceYearMonth  != 201012 and InvoiceYearMonth != 201112")

#Plotting User Monthly Revenue

plot_dfUserMonthlyRevenue = [go.Scatter(x  = df_user_monthly_revenue.query("UserType == 'Existing'")['InvoiceYearMonth'],
                                        y = df_user_monthly_revenue.query("UserType == 'Existing'")['Revenue'],
                                        name = 'Existing'
                                       ),
                             
                             go.Scatter(x  = df_user_monthly_revenue.query("UserType == 'New'")['InvoiceYearMonth'],
                                        y = df_user_monthly_revenue.query("UserType == 'New'")['Revenue'],
                                        name = 'New'
                                       ),
                            ]

layout_dfUserMonthlyRevenue = go.Layout(xaxis = {"type": "category"},
                                       title = 'New vs Existing')

fig6 = go.Figure(data = plot_dfUserMonthlyRevenue, layout =layout_dfUserMonthlyRevenue )

pyoff.iplot(fig6)





In [ ]:
df_user_monthly_revenue.head()

In [ ]:
from jedi import settings
settings.case_insensitive_completion = True

In [ ]:
#create a dataframe that shows new user ratio - we also need to drop NA values (first month new user ratio is 0)

df_user_ratio_Num= df_uk.query("UserType == 'New'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()
df_user_ratio_Din= df_uk.query("UserType == 'Existing'").groupby(['InvoiceYearMonth'])['CustomerID'].nunique()
    
df_user_ratio = (df_user_ratio_Num / df_user_ratio_Din)
df_user_ratio = df_user_ratio.reset_index()
df_user_ratio =df_user_ratio.dropna()
df_user_ratio.head()

In [6]:
# Plotting

plot_UserRatio = [go.Bar( x = df_user_ratio.query("InvoiceYearMonth > 201101 and InvoiceYearMonth < 201112")['InvoiceYearMonth'],
                        y = df_user_ratio.query("InvoiceYearMonth > 201101 and InvoiceYearMonth < 201112")['CustomerID'],
                        )]

layout_UserRatio = go.Layout(xaxis = {"type": "category"},
                            title= "New Customer Ratio")

fig7 = go.Figure(data = plot_UserRatio, layout =layout_UserRatio )
pyoff.iplot(fig7)

NameError: name 'go' is not defined